In [30]:
import pandas as pd
import numpy as np
import os

print("\nIniciando Limpeza de dados")

def ler_e_combinar_csvs(lista_de_caminhos):
    lista_de_dfs = []
    
    for caminho_arquivo in lista_de_caminhos:
        try:
            print(f"Lendo arquivo: {caminho_arquivo}")
            df_temp = pd.read_csv(caminho_arquivo,
                                  encoding='latin-1',
                                  sep=';')
            # CORREÇÃO: Esta linha deve estar dentro do 'try'
            lista_de_dfs.append(df_temp)
        except FileNotFoundError:
            print(f"ERRO: Arquivo não encontrado no caminho: {caminho_arquivo}. Pulando...")
        except Exception as e:
            print(f"ERRO: Ocorreu um erro ao ler o arquivo {caminho_arquivo}: {e}")
            
    if not lista_de_dfs:
        print("Nenhum arquivo CSV foi lido. Retornando com None.")
        return None

    df_combinado = pd.concat(lista_de_dfs, ignore_index=True)
    print("\nConcatenação concluída! DataFrame final criado.")
    return df_combinado        
    
def preencher_nulos_detalhamento(df, coluna_detalhamento):
    #Preenche os valores nulos em uma coluna de detalhamento com uma string padrão.
    df.loc[:, coluna_detalhamento] = df[coluna_detalhamento].fillna('Nenhum registro encontrado')
    return df

def preencher_nulos_documento(df, coluna_documento):
    #Preenche os valores nulos em uma coluna de documento com uma string padrão.
    df.loc[:, coluna_documento] = df[coluna_documento].fillna('s/n')
    return df

def limpar_e_converter_data(df, coluna_data):
    # CORRIGIDO: Esta função agora é a de limpeza de data
    """
    Função que limpa e converte uma coluna de data em um DataFrame.
    """
    df['DATA_TEMP'] = pd.to_datetime(df[coluna_data], format='%d/%m/%Y', errors='coerce')
    print(df['DATA_TEMP'].dtype)

    df[coluna_data] = df['DATA_TEMP']
    df = df.drop('DATA_TEMP', axis=1)
    return df

def limpar_e_converter_valor_reembolsado(df, coluna_valor):
    """
    Limpa e converte uma coluna de valores monetários para tipo numérico
    de forma robusta, garantindo o tipo final float64.
    """
    print(f"\n--- Iniciando a limpeza e conversão da coluna '{coluna_valor}' ---")
    
    # Passo 1: Cria uma cópia da série para evitar problemas
    s = df[coluna_valor].copy()
    
    # Passo 2: Garante que todos os valores são strings para a limpeza funcionar
    s = s.astype(str)
    
    # Passo 3: Limpeza dos caracteres indesejados
    s = (s
         .str.replace('R$', '', regex=False)
         .str.replace('.', '', regex=False)
         .str.replace(',', '.', regex=False)
         .str.strip()  # Remove espaços em branco
        )

    # Passo 4: Substitui explicitamente strings vazias por NaN
    s = s.replace('', np.nan)
    
    # Passo 5: Converte para numérico, tratando erros e forçando o tipo
    s = pd.to_numeric(s, errors='coerce')
    s = s.astype('float64')

    # Passo 6: Atribui a série limpa e convertida de volta ao DataFrame
    df.loc[:, coluna_valor] = s
    
    print(f"\nLimpeza e conversão da coluna '{coluna_valor}' concluída.")
    print(f"Tipo de dado DEPOIS da conversão: {df[coluna_valor].dtype}")

    return df

lista_de_caminhos = ['Dados/despesa_ceaps_2019.csv', 'Dados/despesa_ceaps_2020.csv', 'Dados/despesa_ceaps_2021.csv', 'Dados/despesa_ceaps_2022.csv']

print("Iniciando leitura e combinação dos arquivos...")
df_ceaps = ler_e_combinar_csvs(lista_de_caminhos)

if df_ceaps is not None:
    print("\nDataFrame combinado criado com sucesso! Iniciando a Limpeza...")
    print("\nDataFrame original antes da limpeza:")
    print(df_ceaps.info())

    # ORDEM CORRETA DAS CHAMADAS
    df_ceaps = limpar_e_converter_data(df_ceaps, 'DATA')
    df_ceaps = preencher_nulos_documento(df_ceaps, 'DOCUMENTO')
    df_ceaps = preencher_nulos_detalhamento(df_ceaps, 'DETALHAMENTO')
    df_ceaps = limpar_e_converter_valor_reembolsado(df_ceaps, 'VALOR_REEMBOLSADO')

    print("Pipeline de limpeza concluído!")
    print("\nVisualização do DataFrame final:")
    print(df_ceaps.head())
    print("\nInformações do DataFrame final (verifique os tipos de dados):")
    print(df_ceaps.info())
else:
    print("Falha ao criar o DataFrame. Verifique se os arquivos existem nos caminhos informados.")



Iniciando Limpeza de dados
Iniciando leitura e combinação dos arquivos...
Lendo arquivo: Dados/despesa_ceaps_2019.csv
Lendo arquivo: Dados/despesa_ceaps_2020.csv
Lendo arquivo: Dados/despesa_ceaps_2021.csv
Lendo arquivo: Dados/despesa_ceaps_2022.csv

Concatenação concluída! DataFrame final criado.

DataFrame combinado criado com sucesso! Iniciando a Limpeza...

DataFrame original antes da limpeza:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69356 entries, 0 to 69355
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ANO                69356 non-null  int64 
 1   MES                69356 non-null  int64 
 2   SENADOR            69356 non-null  object
 3   TIPO_DESPESA       69356 non-null  object
 4   CNPJ_CPF           69356 non-null  object
 5   FORNECEDOR         69356 non-null  object
 6   DOCUMENTO          66393 non-null  object
 7   DATA               69356 non-null  object
 8   DETALHAMENTO  